### OAuth2 using PKCE workflow
###### https://www.stefaanlippens.net/oauth-code-flow-pkce.html
###### 

In [1]:
import base64
import hashlib
import html
import json
import os
import re
import urllib.parse
import requests

In [3]:
# keyCloak setup
provider = "http://localhost:9090/realms/master"
provider = "https://fhir.epic.com/interconnect-fhir-oauth"
client_id = "ashok-pkce-test"
username = "ashoksharma"
password = "ashok007"
redirect_uri = "http://localhost:5465/fhir-app"

#### Connect to authentication provider
The first phase of the flow is to connect to the OAuth/OpenID Connect provider and authenticate. For a PKCE-enabled flow we need a some PKCE ingredients from the start.

#### PKCE code verifier and challenge
We need a code verifier, which is a long enough random alphanumeric string, only to be used "client side". We'll use a simple urandom/base64 trick to generate one:

In [4]:
code_verifier = base64.urlsafe_b64encode(os.urandom(40)).decode('utf-8')
print (code_verifier)
code_verifier = re.sub('[^a-zA-Z0-9]+', '', code_verifier)
code_verifier, len(code_verifier)

8GS3mqQwPk2qisUgMoseYh4ujIsZ0M5pc-4OYCAb71PM9e24DJCrSA==


('8GS3mqQwPk2qisUgMoseYh4ujIsZ0M5pc4OYCAb71PM9e24DJCrSA', 53)

To create the PKCE code challenge we hash the code verifier with SHA256 and encode the result in URL-safe base64 (without padding)

In [5]:
code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode('utf-8')
code_challenge = code_challenge.replace('=', '')
code_challenge, len(code_challenge)

('PDtGAxj4vP4X0dSPyT2RP7vP46SOzsZMznBF9ZMCGI4', 43)

#### Request login page
We now have all the pieces for the initial request, which will give us the login page of the authentication provider. Adding the code challenge signals to the OAuth provider that we are expecting the PKCE based flow.

In [7]:
# "/protocol/openid-connect/auth",
state = "fooobarbaz"
resp = requests.get(
    url=provider + "/oauth2/authorize",
    params={
        "response_type": "code",
        "client_id": client_id,
        "scope": "openid",
        "redirect_uri": redirect_uri,
        "state": state,
        "code_challenge": code_challenge,
        "code_challenge_method": "S256",
    },
    allow_redirects=False
)
resp.status_code

200

#### Parse login page (response)
Get cookie data from response headers (requires a bit of manipulation).

In [8]:
cookie = resp.headers['Set-Cookie']
cookie = '; '.join(c.split(';')[0] for c in cookie.split(', '))
cookie

'ASP.NET_SessionId=acmcb2ru1hbmmms1ixjxauty; EpicPersistenceCookie=!U0RKL7YtVnZhTjpFItYMWnabEMGVKH+8PV+XzPCKK7TOWGvokDhQbVFkni+1hJeoIJoEy95bFgsjuOk='

Extract the login URL to post to from the page HTML code. Because the the Keycloak login page is straightforward HTML we can get away with some simple regexes.

In [9]:
page = resp.text
form_action = html.unescape(re.search('<form\s+.*?\s+action="(.*?)"', page, re.DOTALL).group(1))
form_action

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
print(page)

#### Do the login (aka authenticate)
Now, we post the login form with the user we created earlier, passing it the extracted cookie as well.

In [ ]:
resp = requests.post(
    url=form_action, 
    data={
        "username": username,
        "password": password,
    }, 
    headers={"Cookie": cookie},
    allow_redirects=False
)
resp.status_code

In [ ]:
redirect = resp.headers['Location']
redirect

In [ ]:
assert redirect.startswith(redirect_uri)

#### Extract authorization code from redirect
The redirect URL contains the authentication code.

In [ ]:
query = urllib.parse.urlparse(redirect).query
redirect_params = urllib.parse.parse_qs(query)
redirect_params

In [ ]:
auth_code = redirect_params['code'][0]
auth_code

### Exchange authorization code for an access token
We can now exchange the authorization code for an access token. In the normal OAuth authorization flow we should include a static secret here, but instead we provide the code verifier here which acts proof that the initial request was done by us.

In [ ]:
resp = requests.post(
    url=provider + "/protocol/openid-connect/token",
    data={
        "grant_type": "authorization_code",
        "client_id": client_id,
        "redirect_uri": redirect_uri,
        "code": auth_code,
        "code_verifier": code_verifier,
    },
    allow_redirects=False
)
resp.status_code

In the response we get, among others, the access token and id token:

In [ ]:
result = resp.json()
result

### Decode the JWT tokens
The access and id tokens are JWT tokens apparently. Let's decode the payload.

In [ ]:
def _b64_decode(data):
    data += '=' * (4 - len(data) % 4)
    return base64.b64decode(data).decode('utf-8')

def jwt_payload_decode(jwt):
    _, payload, _ = jwt.split('.')
    return json.loads(_b64_decode(payload))

In [ ]:
jwt_payload_decode(result['access_token'])

In [ ]:
jwt_payload_decode(result['id_token'])